# [Toxic comment classification challenge](https://www.kaggle.com/c/jigsaw-toxic-comment-classification-challenge)

In this competition, you’re challenged to build a multi-headed model that’s capable of detecting different types of of toxicity like threats, obscenity, insults, and identity-based hate better than Perspective’s current models. You’ll be using a dataset of comments from Wikipedia’s talk page edits. Improvements to the current model will hopefully help online discussion become more productive and respectful.

In [ ]:
########################################
# Load the packages
########################################
import numpy as np
import pandas as pd
import re

import matplotlib.pyplot as plt
import seaborn as sns

from nltk.stem import SnowballStemmer

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, Embedding, Bidirectional, GRU, Conv1D, GlobalMaxPooling1D, Dropout, TimeDistributed
from keras.layers.merge import concatenate
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint

%matplotlib inline

In [ ]:
########################################
# Define the hyper parameters
########################################
path = 'Dataset/'
TRAIN_DATA_FILE=path + 'train.csv'
TEST_DATA_FILE=path + 'test.csv'

MAX_SEQUENCE_LENGTH = 100
MAX_NB_WORDS = 100000
EMBEDDING_DIM = 50

## Prepare the training / testing data

In [ ]:
########################################
# Load the training / testing set with pandas csv format
########################################
train_df = pd.read_csv(TRAIN_DATA_FILE)
test_df = pd.read_csv(TEST_DATA_FILE)

## Expolary Data Analysis

In [ ]:
print("A quick view of training set")
train_df.head()

In [ ]:
print("A quick view of testing set")
test_df.head()

# Check the labels distribution

### Check the balance of labels

We would like to know the positive ratio of training set. Because we do not want the model become a lazy guy, for a less frequent positive case, we may give it more penalty when model targets it wrong.

In [ ]:
'''
What's the positive ratio of each class ?
'''
def get_pos_ratio(data):
    pass

pos_ratio = []
for col in ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']:
    pos_ratio.append(get_pos_ratio(train_df[col]))

In [ ]:
assert pos_ratio[0] == 0.09584448302009764, "The answer is not correct."
print("Congrats, you passed the test.")

In [ ]:
x = train_df.iloc[:,2:].sum()

plt.figure(figsize=(8,4))
ax= sns.barplot(x.index, x.values, alpha=0.8)
plt.title("# per class")
plt.ylabel('# of Occurrences', fontsize=12)
plt.xlabel('Type ', fontsize=12)

rects = ax.patches
labels = x.values
for rect, label in zip(rects, labels):
    height = rect.get_height()
    ax.text(rect.get_x() + rect.get_width()/2, height + 5, label, ha='center', va='bottom')

plt.show()

### Correlations of labels

Because this is a mulit-label classification, we will want to know the relation betweens labels, which helps for feature engineering and model design. For example, if we know that a toxic comment is always a insult comment, when we have a high confident toxic comment, we can also consider it as insult comment.

In [ ]:
corr=train_df.corr()
plt.figure(figsize=(10,8))
sns.heatmap(corr,
            xticklabels=corr.columns.values,
            yticklabels=corr.columns.values, annot=True)

## Preprocessing

We apply 2 propressing method on texts:

1. Make all alphabet lower case:
    * It is very important. We do not want the model cosider 'Hello' and 'hello' as different words.
2. Remove some special tokens and deal with postfix:
    * For example: what's -> what is, aren't -> are not. Giving the same concept as same token helps for regularization.

Always remember to do preprocess for NLP task. In many kinds of cases, the performance of model trianed with cleaned text significantly outperforms the model trained with raw data. Knowing your data is always the best policy.

In [ ]:
########################################
## Text pre-processing and cleaning
########################################
print('Processing text dataset')
from collections import defaultdict

# regex to remove all Non-Alpha Numeric and space
special_character_removal=re.compile(r'[^a-z\d ]',re.IGNORECASE)

# regex to replace all numeric
replace_numbers=re.compile(r'\d+',re.IGNORECASE)

def clean_text(text, stem_words=False):
    # Clean the text, with the option to remove stopwords and to stem words.
    text = text.lower()
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"i’m", "i am", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"\s{2,}", " ", text)
    text = replace_numbers.sub('', text)
    text = special_character_removal.sub('',text)
    
    return text

In [ ]:
'''
Apply preprocessing and extract the training sentences and testing senteces from pandas dataframe.
Note that there are some N/A comment in the train/test set. Fill them up first.
'''
train_comments = []
test_comments = []

In [ ]:
assert len(train_comments) == 159571 and len(test_comments) == 153164, "It seems that you lost some data."
assert 'E' not in train_comments[0], "It seems you did not preprocess the sentecnes. I found a upper case alphabet in your train set."

### Let's have a comparsion between cleaned text and original one

In [ ]:
for i in range(5):
    print("Cleaned\n", train_comments[i] + '\n')
    print("Raw\n", train_df.iloc[i]['comment_text'] + '\n')
    print("------------------")

# Tokenization

Tokenization separates a sentence into words by space, for example:

* "Hello world" -> ["Hello", "world"]

The input of the neural network is a digit not a word. So we have to apply one hot encoding or index encoding on them.
![onehot](resources/onehot.png)
Now we use keras tokenizer to learn the encoding table.

In [ ]:
# Create a tokenize, which transforms a sentence to a list of ids
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
# Build the relation between words and ids 
tokenizer.fit_on_texts(train_comments + test_comments)

In [ ]:
tokenizer.word_index # map 'the' to 1, map 'to' to 2,......

In [ ]:
# Transform training/testing sentences to training/testing sequences.
train_sequences = tokenizer.texts_to_sequences(train_comments)
test_sequences = tokenizer.texts_to_sequences(test_comments)

## have a look on transformed sequences

In [ ]:
for i in range(1):
    print("Transformed\n", str(train_sequences[i]) + '\n')
    print("Cleaned\n", train_comments[i] + '\n')
    print("------------------")

In [ ]:
word_index = tokenizer.word_index
print('Found %s unique tokens' % len(word_index))

train_data = pad_sequences(train_sequences, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', train_data.shape)
print('Shape of label tensor:', train_labels.shape)

test_data = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of test_data tensor:', test_data.shape)

In [ ]:
'''
Try to build a tokenzier, which transform [['Hello', 'World'], ['Greeting', 'my', 'friend'], ['Hello', 'have', 'a', 'nice', 'day']]
to a list of index sequences. Note that the index should start from 1 because 0 is reserverd for padding token for some framework.
'''
tests_input_sentences =  [['Hello', 'World'], ['Greeting', 'my', 'friend'], ['Hello', 'have', 'a', 'nice', 'day']]
transform_this_sentences = [['Hello', 'my', 'friend']]

def index_encoding(sentences, raw_sent):
    pass
    return results

In [ ]:
transformed = index_encoding(tests_input_sentences, transform_this_sentences)
assert transformed == [[1, 4, 5]], "The answer is not correct."
print("Congrats, you passed the test.")

## Models

## Text RNN

![TextRNN](resources/textrnn.png)

Here we present a classical structure, 2 layer bidirectional GRU, for text classification. Instead of adding a fully connected layer after all time steps, here we only select the last hidden unit of sequence. (LB: 50%, AUC: 0.982)

In [ ]:
########################################
## Define the text rnn model structure
########################################
def get_text_rnn():
    recurrent_units = 48
    dense_units = 32
    output_units = 6
    
    input_layer = Input(shape=(MAX_SEQUENCE_LENGTH,))
    embedding_layer = Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH)(input_layer)
    
    x = Bidirectional(GRU(recurrent_units, return_sequences=True))(embedding_layer)
    x = Bidirectional(GRU(recurrent_units, return_sequences=False))(x)
    
    x = Dense(dense_units, activation="relu")(x)
    output_layer = Dense(output_units, activation="sigmoid")(x)
    
    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

## TextCNN

![TextCNN](resources/textcnn.png)

Convolution in natural langauge proceessing can be consider as a special type of ngram. We simply select the kernels with window sizes (2, 3, 4) to extract regional features. (LB: 50%, AUC: 0.982)

In [ ]:
########################################
## Define the text cnn model structure
########################################
def get_text_cnn():
    filter_nums = 120
    dense_units = 72
    output_units = 6
    
    input_layer = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
    embedding_layer = Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH,)(input_layer)
        
    conv_0 = Conv1D(filter_nums, 2, kernel_initializer="normal", padding="valid", activation="relu")(embedding_layer)
    conv_1 = Conv1D(filter_nums, 3, kernel_initializer="normal", padding="valid", activation="relu")(embedding_layer)
    conv_2 = Conv1D(filter_nums, 4, kernel_initializer="normal", padding="valid", activation="relu")(embedding_layer)

    maxpool_0 = GlobalMaxPooling1D()(conv_0)
    maxpool_1 = GlobalMaxPooling1D()(conv_1)
    maxpool_2 = GlobalMaxPooling1D()(conv_2)

    merged_tensor = concatenate([maxpool_0, maxpool_1, maxpool_2])
    h1 = Dense(units=dense_units, activation="relu")(merged_tensor)
    output = Dense(units=output_units, activation='sigmoid')(h1)

    model = Model(inputs=input_layer, outputs=output)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

## Hybrid Text NN

![hybrid](resources/hybrid.png)

This structure mixed the feature representation ideas of RNN and CNN. We firstly place the recurrent layer after embedding for building the word's level sequential information and make it connected with a convolution layer to extract the regional features of hiddens. (LB: 30%, AUC: 0.983)

In [ ]:
########################################
## Define the text hybrid model structure
########################################
def get_hybrid_textnn():
    recurrent_units = 48
    dense_units = 32
    filter_nums = 64
    output_units = 6

    input_layer = Input(shape=(MAX_SEQUENCE_LENGTH,))
    embedding_layer = Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH)(input_layer)
    
    x = Bidirectional(GRU(recurrent_units, return_sequences=True))(embedding_layer)
    x = Conv1D(filter_nums, 2, kernel_initializer="normal", padding="valid", activation="relu")(x)    
    
    max_pool = GlobalMaxPooling1D()(x)
    max_pool = Dropout(0.5)(max_pool)
    
    output_layer = Dense(output_units, activation="sigmoid")(max_pool)

    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

### K-Fold Cross Valiadtion
![kfold](resources/kfold.png)

In [ ]:
########################################
## Construct the cross-validation framework
########################################
def _train_model_by_logloss(model, batch_size, train_x, train_y, val_x, val_y, fold_id):
    # set an early stopping checker.
    # the training phase would stop when validation log loss decreases continuously for `patience` rounds. 
    early_stopping = EarlyStopping(monitor='val_loss', patience=5)
    bst_model_path = "ToxicModel" + str(fold_id) + '.h5'
    model_checkpoint = ModelCheckpoint(bst_model_path, save_best_only=True, save_weights_only=True)
    
    # training on given fold data
    hist = model.fit(train_x, train_y,
        validation_data=(val_x, val_y),
        epochs=50, batch_size=batch_size, shuffle=True,
        callbacks=[early_stopping, model_checkpoint])
    
    # get the minimal validation log loss on this fold
    bst_val_score = min(hist.history['val_loss'])

    # return the model with best weight, best fold-val score
    return model, bst_val_score

def train_folds(X, y, fold_count, batch_size, get_model_func):
    fold_size = len(X) // fold_count
    models = []
    score = 0
    
    # split the whole dataset to `fold_count` fold, and train our model on each fold
    for fold_id in range(0, fold_count):
        fold_start = fold_size * fold_id
        fold_end = fold_start + fold_size

        if fold_id == fold_size - 1:
            fold_end = len(X)

        # Generate the train/val data on fold i
        train_x = np.concatenate([X[:fold_start], X[fold_end:]])
        train_y = np.concatenate([y[:fold_start], y[fold_end:]])

        val_x = X[fold_start:fold_end]
        val_y = y[fold_start:fold_end]
    
        print("Training on fold #", fold_id)
        model, bst_val_score = _train_model_by_logloss(get_model_func(), batch_size, train_x, train_y, val_x, val_y, fold_id)
        score += bst_val_score
        models.append(model)
    return models, score / fold_count

In [ ]:
models, val_loss = train_folds(train_data, train_labels, 2, 256, get_text_cnn)

# Practice: Try to beat the baseline: val_loss=0.050

In [ ]:
your_batch_size = 256

def get_your_model():
    '''your show time'''
    return model

In [ ]:
models, val_loss = train_folds(train_data, train_labels, 2, your_batch_size, get_your_model)

# Make the predections

In [ ]:
#test_data = test_df
CLASSES = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
submit_path_prefix = "ToxicNN-" + str(MAX_SEQUENCE_LENGTH) 

print("Predicting testing results...")
test_predicts_list = []
for fold_id, model in enumerate(models):
    test_predicts = model.predict(test_data, batch_size=256, verbose=1)
    test_predicts_list.append(test_predicts)

# merge each folds' predictions by averaging
test_predicts = np.zeros(test_predicts_list[0].shape)
for fold_predict in test_predicts_list:
    test_predicts += fold_predict
test_predicts /= len(test_predicts_list)

# create the submission file
test_ids = test_df["id"].values
test_ids = test_ids.reshape((len(test_ids), 1))
test_predicts = pd.DataFrame(data=test_predicts, columns=CLASSES)
test_predicts["id"] = test_ids
test_predicts = test_predicts[["id"] + CLASSES]
submit_path = submit_path_prefix + "-Loss{:4f}.csv".format(val_loss)
test_predicts.to_csv(submit_path, index=False)

## Discussions


### Better method to compose a sequence of vectors into a single vector ?

Either in CNN or RNN, the outputs are a sequence of vectors, which means a best practice to compose a sequence of vectors into a single one is very important. We have tried to simply select the last one (in RNN) or select the one with max value (in CNN and hybrid NN) to represent a sequence, and there clearly is much room for improvement. For example, how about selecting the top K max vectors? or averaging the whole sequence to get one vector? Furthermore, we can apply **weighted averaging** to the sequence, which is called **Attention** in natural language processing  and it does help a lot on catching information in long sequences.

### Jointly train or not ?

This is a multilabel classification challenge, so why do we jointly train 6 labels together rather than train them one by one? Indeed, this is a good question. In some cases which are labeled sparsely or not clearly related to other classes (like threat in this dataset), training these labels independently might get a better socre because these cases should build their own unique feature representations. You can give it a try, and find the best combination on training labels.

### The power of unsupervised learing

In the above tutorial, we just defined a **random initialized** embedding matrix for text classificaiton. With this method, the embedding matrix will fit well on the training set but it would also be biasd by some special tokens or noise since our dataset is not that large, that cause overfitting.

We can deal with this by using some pretrained resources, like:

* [GloVe embedding](https://nlp.stanford.edu/projects/glove/)
* [Fasttext embedding](https://github.com/facebookresearch/fastText/blob/master/pretrained-vectors.md)
* [Word2Vec](https://radimrehurek.com/gensim/models/word2vec.html)

You might get a significant boost by replacing the old matrix with vectors pretrained on a big corpus which catch the similarity between words.

In [ ]:
'''
Try to load the word embedding as the format
{
   'word1': embedding,
   'word2': embedding,
   ...
}
i.e. A key-value pair whose key is the word and the whose value is the embedding
'''
GLOVE_EMBEDDING = {}

def load_embedding(embeddings_index, embedding_path='glove.6B.50d.txt'):
    '''return a dict whose key is word, value is pretrained word embedding'''
    return embeddings_index

GLOVE_EMBEDDING = load_embedding(GLOVE_EMBEDDING)

In [ ]:
assert len(GLOVE_EMBEDDING) == 400000 , "Failed. Did you load the whole file ?"
assert 'hello' in GLOVE_EMBEDDING.keys(), "Oops, it seems that you miss some words"
assert len(GLOVE_EMBEDDING['hello'] == 50), "You have a wrong dimension. Check it again."

print("Congrats, you passed the test.")

In [ ]:
def build_embedding_matrix(embeddings_index, word_index):
    embedding_matrix = np.zeros((MAX_NB_WORDS, 50))

    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if i >= MAX_NB_WORDS:
            continue
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    print('Null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))
    return embedding_matrix

GLOVE_EMBEDDING = build_embedding_matrix(GLOVE_EMBEDDING, word_index)

## Replace the embedding to a pretrained one is easy

To freeze or keep train the word embedding depends on your task. Sometimes, we tend to add a WordEncoder among the word embedding.
The WordEncoder can be a simple Dense layer or something complicated (for example: Highway network).

```
embedding_layer = Embedding(MAX_NB_WORDS,
                        EMBEDDING_DIM,
                        weights=[GLOVE_EMBEDDING],
                        input_length=MAX_SEQUENCE_LENGTH,
                        trainable=False)(TENSOR)
```

In [ ]:
'''
Try to create a model with pretrained word embedding which has a single layer dense WordEncoder.
Reference: https://keras.io/layers/wrappers/
'''
def get_your_model_with_pretrained_embedding():
    '''your show time'''
    return model

In [ ]:
assert type(get_your_model_with_pretrained_embedding().layers[2]) == TimeDistributed, "Your model do not have a word encoder."
print("Congrats, you passed the test.")

In [ ]:
models, val_loss = train_folds(train_data, train_labels, 2, 256, get_your_model_with_pretrained_embedding)

# Reference: [My solution to this challenge](https://github.com/zake7749/DeepToxic)